In [ ]:
from sentinelhub import (
    CRS,
    BBox,
    BBoxSplitter,
    bbox_to_dimensions,
    DataCollection,
    MimeType,
    MosaickingOrder,
    SentinelHubCatalog,
    SentinelHubDownloadClient,
    SentinelHubRequest,
    TileSplitter,
    read_data,
    SHConfig
)



import os
import numpy as np
from shapely.geometry import shape

import geopandas as gpd
import rasterio as rio
from rasterio import features
from rasterio.merge import merge
import math

from dotenv import load_dotenv
from evalScripts_download import evalscripts

In [13]:
load_dotenv()
# authenticate to SH using client and client secret
config = SHConfig()

config.sh_client_id = os.getenv("SH_CLIENT_ID")
config.sh_client_secret = os.getenv("SH_CLIENT_SECRET")

catalog = SentinelHubCatalog(config=config)

In [ ]:
input_file_path = r"geometry/aoi_3.json"
start_date = "2024-08-01"
end_date = "2024-08-15" 
resolution = 10
indice = "NDBI"
evalscript = evalscripts[indice]

In [27]:
def loadAOI (aoipath, resolution):
    geojson = read_data(aoipath)
    aoi = shape(geojson["features"][0]["geometry"])
    aoibbox = BBox(bbox = aoi.bounds, crs = CRS.WGS84)
    aoisize = bbox_to_dimensions(aoibbox, resolution= resolution)
    
    return [aoisize,aoi]
    

In [28]:
aoi = loadAOI(input_file_path,resolution)

In [29]:
def calculateGridSize(bbox_size):
    width, height = bbox_size

    # Calculate the number of tiles needed for width and height
    tiles_x = math.ceil(width / 1200)
    tiles_y = math.ceil(height / 1200)

    return (tiles_x, tiles_y)

In [30]:
gridsize = calculateGridSize(aoi[0])
bbox_splitter = BBoxSplitter([aoi[1]], CRS.WGS84, gridsize, reduce_bbox_sizes=True)

In [31]:
output_folder = r"results"
if not os.path.exists(f"{output_folder}"):
    os.makedirs(f"{output_folder}")

In [32]:
def get_request_by_subarea(bbox, evalscript, start_date, end_date):
    size = bbox_to_dimensions(bbox, resolution=resolution)
    return SentinelHubRequest(
        evalscript=evalscript,
        input_data=[
            SentinelHubRequest.input_data(
                data_collection=DataCollection.SENTINEL2_L2A,
                time_interval=(start_date, end_date),
            )
        ],
        responses=[SentinelHubRequest.output_response("default", MimeType.TIFF)],
        bbox=bbox,
        size=size,
        data_folder=f"{output_folder}",
        config=config,
    )

In [33]:
bbox_list = bbox_splitter.get_bbox_list()
sh_requests_deforestation = [get_request_by_subarea(bbox, evalscript, start_date, end_date) for bbox in bbox_list]
dl_requests = [request.download_list[0] for request in sh_requests_deforestation]

# download data with multiple threads
downloaded_data = SentinelHubDownloadClient(config=config).download(dl_requests, max_threads=5)

In [34]:
tiffiles = []
for folder, subfolder, files in os.walk(output_folder):
    for file in files:
        if file.lower().endswith((".tif",".tiff")):
            tiffiles.append(os.path.join(folder, file))

In [35]:
def generateMosaic(tiffiles, index):
    mosaicFiles =  [rio.open(file) for file in tiffiles]

# Hacer el mosaico
    mosaic, out_trans = merge(mosaicFiles)

# Copiar metadatos del primero
    out_meta = mosaicFiles[0].meta.copy()

# Actualizar metadatos con nueva dimensión y transform
    out_meta.update({
    "driver": "GTiff",
    "height": mosaic.shape[1],
    "width": mosaic.shape[2],
    "transform": out_trans
})

#guardar msoaico
    with rio.open(os.path.join(output_folder,"mosaico_{}.tif".format(index)), "w", **out_meta) as dest:
        dest.write(mosaic)

generateMosaic(tiffiles,indice)

In [36]:
with rio.open(os.path.join(output_folder,"mosaico_{}.tif".format(indice.lower()))) as src:
    raster_data = src.read(1)  # primera banda
    transform = src.transform
    crs = src.crs
    width, height = src.width, src.height
    bounds = src.bounds

In [37]:
def reclassifyRaster(raster_data, indice, output_folder):
    
    reclasification_methods = {
        "ndvi": lambda x: np.where(np.isnan(x), np.nan,
                                   np.where(x < 0, 1,
                                            np.where(x <= 0.2, 2,
                                                     np.where(x <= 0.5, 3,
                                                              np.where(x <= 0.8, 4, 5))))),
        "gndvi": lambda x: np.where(np.isnan(x), np.nan,
                                    np.where(x < 0, 1,
                                             np.where(x <= 0.5, 2,
                                                      np.where(x <= 0.8, 3, 4)))),
        "evi": lambda x: np.where(np.isnan(x), np.nan,
                                  np.where(x < 0, 1,
                                           np.where(x <= 0.5, 2,
                                                    np.where(x <= 0.8, 3, 4)))),
        "savi": lambda x: np.where(np.isnan(x), np.nan,
                                   np.where(x < 0, 1,
                                            np.where(x <= 0.5, 2,
                                                     np.where(x <= 0.8, 3, 4)))),
        "ndci": lambda x: np.where(np.isnan(x), np.nan,
                                   np.where(x < 0, 1,
                                            np.where(x <= 0.2, 2, 3))),
        "ndmi": lambda x: np.where(np.isnan(x), np.nan,
                                   np.where(x < 0, 1,
                                            np.where(x <= 0.5, 2,
                                                     np.where(x <= 0.8, 3, 4)))),
        "ndsi": lambda x: np.where(np.isnan(x), np.nan,
                                   np.where(x < 0, 1,
                                            np.where(x <= 0.45, 2, 3))),
        "ndwi": lambda x: np.where(np.isnan(x), np.nan,
                                   np.where(x < 0, 1, 2)),
        "ndbi": lambda x: np.where(np.isnan(x), np.nan,
                                   np.where(x <= -0.05, 1,
                                            np.where(x <= 0.05, 2, 3))),
        "nbr": lambda x: np.where(np.isnan(x), np.nan,
                                  np.where(x <= 0.1, 1,
                                           np.where(x <= 0.44, 2,
                                                    np.where(x <= 0.66, 3, 4)))),
    }
    
    combined_state = {
    "ndvi": {
        1: ["Ausencia de vegetación o cobertura vegetal muy escasa", "< 0"],
        2: ["Vegetación escasa o estresada", "0 - 0.2"],
        3: ["Vegetación en condiciones moderadas", "0.2 - 0.5"],
        4: ["Vegetación densa y saludable", "0.5 - 0.8"],
        5: ["Vegetación extremadamente densa y saludable", "0.8 - 1"],
    },
    "gndvi": {
        1: ["Áreas con baja o nula vegetación, suelos desnudos, agua o superficies construidas", "< 0"],
        2: ["Vegetación en condiciones saludables, pero no particularmente densa", "0 - 0.5"],
        3: ["Vegetación densa y saludable", "0.5 - 0.8"],
        4: ["Vegetación extremadamente densa y saludable", "0.8 - 1"]
    },
    "evi": {
        1: ["Zonas con muy poca o ninguna vegetacion", "< 0"],
        2: ["Presencia creciente de vegetacion", "0 - 0.5"],
        3: ["Vegetacion mas densa y saludable que las areas verdes", "0.5 - 0.8"],
        4: ["Vegetacion con la maxima densidad y salud de la vegetación", "0.8 - 1"],
    },
    "savi": {
        1: ["Muy baja vegetacion o suelo expuesto", "< 0"],
        2: ["Vegetación escasa", "0 - 0.5"],
        3: ["Vegetación moderada", "0.5 - 0.8"],
        4: ["Vegetación extremadamente densa", "0.8 - 1"]
    },
    "ndci": {
        1: ["Vegetacion con baja concentacion de clorofila", "< 0"],
        2: ["Vegetación con concentracion de clorofila moderada", "0 - 0.2"],
        3: ["Vegetacion con alta concentacion de clorofila", "0.2 - 1"],
    },
    "ndmi": {
        1: ["Baja humedad en la vegetación", "< 0"],
        2: ["Moderada humedad en la vegetacion", "0 - 0.5"],
        3: ["Alta humedad en la vegetación", "0.5 - 0.8"],
        4: ["Muy alta humedad en la vegetación", "0.8 - 1"],
    },
    "ndsi": {
        1: ["Zonas Ausencia de Nieve", "< 0"],
        2: ["Zonas con poca nieve", "0 - 0.4"],
        3: ["Zonas con nieve densa", "0.4 - 1"]
    },
    "ndwi": {
        1: ["Zonas sin presencia de agua", "< 0"],
        2: ["Zonas inundadas", "0 - 1"],
    },
    "ndbi": {
        1: ["Baja presencia de superficies construidas o alta presencia de vegetación", "< -0.05"],
        2: ["Zonas construidas con vegetación", "-0.05 - 0.05"],
        3: ["Zonas con alta construccion", "0.05 - 1"],
    },
    "nbr": {
        1: ["Zonas no quemadas", "< 0.1"],
        2: ["Zonas quemadas con gravedad moderada baja", "0.1 - 0.44"],
        3: ["Zonas quemadas con gravedad moderada alta", "0.44 - 0.66"],
        4: ["Zonas gravemente quemadas", "0.66 - 1"]
    }
}

    def getClassification(index: str ):
        #Devuelve la clasificación de estados para el índice solicitado.
        return combined_state.get(index.lower(), combined_state["nbr"]) 
    
    #Obtencion metodo de reclasificacion
    method = reclasification_methods[indice.lower()]
    reclasificado = method(raster_data)
    
    #identificaci[on de valores unicos]
    unique_vals = np.unique(reclasificado[~np.isnan(reclasificado)])
    IndexGroups = {int(val): [] for val in unique_vals}
    
    for val in unique_vals:
        vals = raster_data[reclasificado == val]
        IndexGroups[int(val)] = vals.tolist()
        
    mask = ~np.isnan(reclasificado)
    
    #generacion de poligonos
    polygons = []
    for geom, value in features.shapes(reclasificado.astype(np.int16),mask=mask,transform=transform):
        if value == 0:
            continue
        polygons.append({"geometry": shape(geom), "value": int(value)})

    gdf = gpd.GeoDataFrame(polygons, crs=crs)
    
    #disolucion de poligonos a partir del value
    dissolved = gdf.dissolve(by="value", as_index=False)
    
    #definicion de atributos de estado, rango de indice y media de rango

    stateClasification = getClassification(indice)

    dissolved["IndexState"] = dissolved["value"].apply(lambda v: stateClasification.get(int(v), ["Sin clasificacion"])[0])
    dissolved["IndexRange"] = dissolved["value"].apply(lambda v: stateClasification.get(int(v), ["", "Sin clasificacion"])[1])
    dissolved["IndexRangeMean"] = dissolved["value"].apply(lambda v: round(sum(IndexGroups.get(int(v), [])) / len(IndexGroups.get(int(v), [])), 4)
        if IndexGroups.get(int(v)) else None
    )
    
    #convertir a wgs84
    dissolved = dissolved.to_crs(epsg=4326)
    
    #guardado del archivo
    dissolved.to_file(os.path.join(output_folder,"indice_{}_reclasificado.geojson".format(indice.lower())), driver="GeoJSON")

reclassifyRaster(raster_data,indice, output_folder)

    